<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install workalendar
!pip install pytz


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 26.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.9 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732000 sha256=2314f2570a1566c081c059a487a287386eb05c0bbd7996619dfb3902774fd759
  Stored in directory: /root/.cache/pip/wheels/8f/bd/f9/5c4c39b529e0322b08979e1c465e203218bc2cca75d20f7df5
Successfully built pymeeus


In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Cargar acumulativo histórico

In [104]:
import os
import glob
import pandas as pd
from datetime import datetime

# Ruta base donde están los históricos
ruta_base = "/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico"
os.makedirs(ruta_base, exist_ok=True)

# Buscar el último archivo guardado
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

if archivos:
    archivo_historico = archivos[-1]
    df_historico = pd.read_excel(archivo_historico)
    print(f"✅ Último histórico cargado desde: {archivo_historico}")
else:
    df_historico = pd.DataFrame()
    print("⚠️ No hay acumulativos anteriores. Se parte de vacío.")


✅ Último histórico cargado desde: /content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico/Acumulativo_Recuento_historico_2025-05-14_2.xlsx


# **Extraer Canales Realizados y crear Hoja REGISTRO REALIZADA**

# Teléfono Realizada

In [105]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dftelefono = pd.read_excel(file_path, sheet_name='Teléfono')


dftelefono.head()

,PERSONA,LOGIN,EMPRESA,TELÉFONO,HORARIO,ESCENARIO,REALIZADA
0,PERSONA 1,NaN,ACER 01,914192640,NaN,Chromebook: ¿Cómo puedo recuperar mi Chromeboo...,ok
1,PERSONA 2,NaN,ACER 02,914192640,NaN,Chromebook: ¿Cómo reinstalar Chorme OS?,ok
2,PERSONA 3,NaN,ACER 03,914192640,NaN,Chromebook: ¿Cómo habilitar la carga adaptativa?,ok
3,PERSONA 4,NaN,ACER 04,914192640,NaN,Chromebook: ¿Cómo puedo ver que versión de Chr...,ok
4,PERSONA 5,NaN,ADESLAS 01,900505040,NaN,¿Qué modalidades de seguros de salud ofrecéis?,ok


**Contar previamente la cantidad de ok y no ok presentes**

In [106]:
conteo = dftelefono['REALIZADA'].str.strip().str.lower()
total_ok_no_ok = (conteo == 'ok').sum() + (conteo == 'no ok').sum()
print(f"Total de 'ok' y 'no ok': {total_ok_no_ok}")


Total de 'ok' y 'no ok': 140


In [107]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dftelefono['EMPRESA'] = dftelefono['EMPRESA'].str.strip()

dftelefono['EMPRESA'] = (dftelefono['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar Hoja Registro para Realizada Teléfono**

In [108]:
# Limpiar espacios adelante y atrás en todas las columnas de tipo texto
dftelefono = dftelefono.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Estandarizar columna 'REALIZADA'
dftelefono['realizada_limpia'] = dftelefono['REALIZADA'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dftelefono[dftelefono['realizada_limpia'].isin(['ok', 'no ok'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Telefono'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dftelefono = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dftelefono = dftelefono[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dftelefono.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dftelefono.head()


<ipython-input-108-f55b5caaf17a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['tipo_contacto'] = 'Telefono'
<ipython-input-108-f55b5caaf17a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['tipo_de_acción'] = 'Realizada'


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Telefono,Realizada,4
1,adeslas,Telefono,Realizada,1
2,aeda,Telefono,Realizada,2
3,asisa,Telefono,Realizada,1
4,asus,Telefono,Realizada,1


# Mail Realizada

In [109]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfmail = pd.read_excel(file_path, sheet_name='Mail')


dfmail.head()

,EMPRESA,ENLACE,NUM ESCENARIO,ESCENARIO,NOMBRE,MAIL,DIRECCIÓN,TELÉFONO,FECHA NACIMIENTO,DNI,ESTADO CORREO
0,ACER,https://es.answers.acer.com/app/ask,ACER 01,Chromebook: ¿Cómo puedo recuperar mi Chromeboo...,Adrián Romo Miró,adrirommi59@hotmail.com,C/Ferrocarril 17 3º 6ª 45006 Toledo,615628741,1959-01-19,40562415Y,enviado
1,AEDA,info@aedashomes.com,AEDA 01,¿Tienen opciones de financiación o acuerdos co...,Adrián Romo Miró,adrirommi59@hotmail.com,C/Ferrocarril 17 3º 6ª 45006 Toledo,615628741,1959-01-19,40562415Y,enviado
2,BOSCH CAR SERVICE,atencion_boschcarservice@es.bosch.com,BOSCH CAR 01,"Tengo que hacerle una revisión a mi coche, ¿po...",Adrián Romo Miró,adrirommi59@hotmail.com,C/Ferrocarril 17 3º 6ª 45006 Toledo,615628741,1959-01-19,40562415Y,enviado
3,BRUNEAU,pedidos@bruneau.es,BRUNEAU 01,"Hola, quisiera darme de alta con vosotros y qu...",Adrián Romo Miró,adrirommi59@hotmail.com,C/Ferrocarril 17 3º 6ª 45006 Toledo,615628741,1959-01-19,40562415Y,enviado
4,CANAL ISABEL II,clientes@canaldeisabelsegunda.es,CANAL ISABEL II 01,¿Me podéis indicar la dirección de alguna ofic...,Adrián Romo Miró,adrirommi59@hotmail.com,C/Ferrocarril 17 3º 6ª 45006 Toledo,615628741,1959-01-19,40562415Y,enviado


**Contar la cantidad de enviado**

In [110]:
enviados = (dfmail['ESTADO CORREO'].str.strip().str.lower() == 'enviado').sum()
print(f"Cantidad de 'enviado': {enviados}")

Cantidad de 'enviado': 25


**Pasar todos los datos de Empresa a minúscula**

In [111]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfmail['EMPRESA'] = dfmail['EMPRESA'].str.strip()

dfmail['EMPRESA'] = (dfmail['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

**Generar hoja Registro de Realizadas Mail**

In [112]:
# Estandarizar columna 'REALIZADA'
dfmail['realizada_limpia'] = dfmail['ESTADO CORREO'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfmail[dfmail['realizada_limpia'].isin(['enviado'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Mail/formulario'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfmail = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfmail = dfmail[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfmail.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfmail.head()

<ipython-input-112-e563573999f8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['tipo_contacto'] = 'Mail/formulario'
<ipython-input-112-e563573999f8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['tipo_de_acción'] = 'Realizada'


,empresa,tipo_contacto,tipo_de_acción,cantidad
0,acer,Mail/formulario,Realizada,1
1,aeda,Mail/formulario,Realizada,1
2,bosch car service,Mail/formulario,Realizada,1
3,bruneau,Mail/formulario,Realizada,1
4,canal isabel ii,Mail/formulario,Realizada,1


# WEB Realizada

In [113]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfweb = pd.read_excel(file_path, sheet_name='Web')


dfweb.head()

,EMPRESA,ENLACE,NUM ESCENARIO,ESCENARIO,PLATAFORMA,ESTADO NAVEGACIÓN


In [114]:
enviados = (dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower() == 'encontrada').sum()
print(f"Cantidad de 'encontrada': {enviados}")

Cantidad de 'encontrada': 0


In [115]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
dfweb['EMPRESA'] = dfweb['EMPRESA'].str.strip()

dfweb['EMPRESA'] = (dfweb['EMPRESA'].str.lower().str.replace(r'\d+', '', regex=True))

In [116]:
# Estandarizar columna 'REALIZADA'
dfweb['realizada_limpia'] = dfweb['ESTADO NAVEGACIÓN'].str.strip().str.lower()

# Filtrar solo filas con 'ok' o 'no ok'
df_filtrado = dfweb[dfweb['realizada_limpia'].isin(['encontrada'])]

# Agregar columna fija
df_filtrado['tipo_contacto'] = 'Web'
df_filtrado['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
dfweb = df_filtrado.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Reordenar columnas si querés
dfweb = dfweb[['EMPRESA', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

dfweb.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar resultado
dfweb.head()

,empresa,tipo_contacto,tipo_de_acción,cantidad


# Facebook y Twitter Realizada

In [117]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfredes = pd.read_excel(file_path, sheet_name='Redes')


dfredes.head()

,EMPRESA,ENLACE RR SS,NUM ESCENARIO,ESCENARIO,RR SS A UTILIZAR,CUENTA RR SS,ESTADO RR SS,tipo_contacto


**Recuento de la cantidad por canal**

In [118]:
# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].astype(str).str.strip().str.lower()
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].fillna('').str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo los "enviado"
df_enviados = dfredes[dfredes['estado_rr_ss_limpio'] == 'enviado']

# Verificamos si el DataFrame tiene datos antes de proceder
if not df_enviados.empty:
    conteo = df_enviados['tipo_contacto'].value_counts().loc[['Facebook', 'Twitter']].to_dict()
    print(f"Cantidad de 'enviado': {conteo}")
else:
    print("No hay datos disponibles en el DataFrame para procesar.")

# Mostramos resultado
print(f"Cantidad de 'enviado': {conteo}")


No hay datos disponibles en el DataFrame para procesar.
Cantidad de 'enviado': 0       ok
1       ok
2       ok
3       ok
4       ok
      ... 
150    NaN
151    NaN
152    NaN
153    NaN
154    NaN
Name: REALIZADA, Length: 155, dtype: object


**Ejecutar para Facebook**

In [119]:
# Estandarizamos texto
dfredes['estado_rr_ss_limpio'] = dfredes['ESTADO RR SS'].str.strip().str.lower()
dfredes['tipo_contacto'] = dfredes['tipo_contacto'].str.strip().str.capitalize()  # o 'tipo_contacto' si es otro nombre

# Filtramos solo "enviado" por canal
df_facebook = dfredes[(dfredes['estado_rr_ss_limpio'] == 'enviado') & (dfredes['tipo_contacto'] == 'Facebook')]

# Agregar columnas fijas
for df_temp in [df_facebook]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_facebook = df_facebook.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_facebook.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

# Mostrar ambos resultados
print("📘 Facebook:")
df_facebook.head()


📘 Facebook:


,empresa,tipo_contacto,tipo_de_acción,cantidad


In [120]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_facebook['empresa'] = df_facebook['empresa'].str.strip()

df_facebook['empresa'] = (df_facebook['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Ejecutar para Twitter**

In [121]:
# Filtramos solo "enviado" por canal
df_twitter  = dfredes[(dfredes['estado_rr_ss_limpio'] == 'enviado') & (dfredes['tipo_contacto'] == 'Twitter')]

# Agregar columnas fijas
for df_temp in [df_twitter]:
    df_temp['tipo_de_acción'] = 'Realizada'
    df_temp['tipo_contacto'] = df_temp['tipo_contacto']

# Agrupar y contar
df_twitter  = df_twitter.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_twitter.rename(columns={'EMPRESA': 'empresa'}, inplace=True)

print("\n Twitter:")
df_twitter.head()


 Twitter:


,empresa,tipo_contacto,tipo_de_acción,cantidad


In [122]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_twitter['empresa'] = df_twitter['empresa'].str.strip()

df_twitter['empresa'] = (df_twitter['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Chat y WhatsApp Realizada

In [123]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Realizada todos los canales.xlsx'

# Cambia 'nombre_archivo.xlsx' por el nombre real del archivo que subiste
dfchats = pd.read_excel(file_path, sheet_name='Chats')


dfchats.head()

,EMPRESA,ENLACE/NÚMERO,NUM ESCENARIO,ESCENARIO,PLATAFORMA,ESTADO CHAT,tipo_contacto


**Contabilizar la cantidad por canal de realizadas**

In [124]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo los "enviado"
df_enviados = dfchats[dfchats['estado_chat_limpio'] == 'atendido']

# Verifica si las categorías están presentes en la columna 'tipo_contacto'
categorias_validas = ['Whatsapp', 'Chat']
categorias_actuales = df_enviados['tipo_contacto'].unique()

# Comprobar si todas las categorías existen en el DataFrame
print(f"Categorías actuales en 'tipo_contacto': {categorias_actuales}")

# Si las categorías están presentes, realizamos el conteo
if all(categoria in categorias_actuales for categoria in categorias_validas):
    conteo = df_enviados['tipo_contacto'].value_counts().loc[categorias_validas].to_dict()
    print(f"Cantidad de 'enviado': {conteo}")
else:
    print("Algunas categorías no están presentes en 'tipo_contacto'.")


Categorías actuales en 'tipo_contacto': []
Algunas categorías no están presentes en 'tipo_contacto'.


**Crear hoja Registro para WhatsApp**

In [125]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_wapp = dfchats[(dfchats['estado_chat_limpio'] == 'atendido') & (dfchats['tipo_contacto'] == 'Whatsapp')]

# Agregar columnas fijas directamente a df_wapp
df_wapp['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_wapp = df_wapp.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_wapp.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_wapp['tipo_contacto'] = df_wapp['tipo_contacto'].replace('Whatsapp', 'WhatsApp')


# Mostrar resultado
print("📘 WhatsApp:")
df_wapp.head()

📘 WhatsApp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


Pasar las empresas a minúscula

In [126]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_wapp['empresa'] = df_wapp['empresa'].str.strip()

df_wapp['empresa'] = (df_wapp['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

**Crear hoja Chat Web para realizadas**

In [127]:
# Estandarizamos texto
dfchats['estado_chat_limpio'] = dfchats['ESTADO CHAT'].str.strip().str.lower()
dfchats['tipo_contacto'] = dfchats['tipo_contacto'].str.strip().str.capitalize()

# Filtramos solo "enviado" por canal
df_chat = dfchats[(dfchats['estado_chat_limpio'] == 'atendido') & (dfchats['tipo_contacto'] == 'Chat')]

# Agregar columnas fijas directamente a df_wapp
df_chat['tipo_de_acción'] = 'Realizada'

# Agrupar y contar
df_chat = df_chat.groupby(['EMPRESA', 'tipo_contacto', 'tipo_de_acción']).size().reset_index(name='cantidad')

# Renombrar columna 'EMPRESA' a 'empresa' si querés estandarizar
df_chat.rename(columns={'EMPRESA': 'empresa'}, inplace=True)
df_chat['tipo_contacto'] = df_chat['tipo_contacto'].replace('Chat', 'Chat')

# Mostrar resultado
print("📘 WhatsApp:")
df_chat.head()

📘 WhatsApp:


,empresa,tipo_contacto,tipo_de_acción,cantidad


Pasar nombre empresa a minúscula

In [128]:
# Limpia los nombres de columna: quita espacios, baja a minúsculas
df_chat['empresa'] = df_chat['empresa'].str.strip()

df_chat['empresa'] = (df_chat['empresa'].str.lower().str.replace(r'\d+', '', regex=True))

# Agrupar todos los canales Realizada

In [79]:
# === UNIÓN FINAL DE TODOS LOS CANALES ===
df_final_realizada = pd.concat([dftelefono, dfmail, df_facebook, df_twitter, df_chat, df_wapp, dfweb], ignore_index=True)
df_final_realizada = df_final_realizada[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostrar resultado
print(df_final_realizada)

              empresa    tipo_contacto tipo_de_acción  cantidad
0                acer         Telefono      Realizada         4
1             adeslas         Telefono      Realizada         1
2                aeda         Telefono      Realizada         2
3               asisa         Telefono      Realizada         1
4                asus         Telefono      Realizada         1
..                ...              ...            ...       ...
85            naturgy  Mail/formulario      Realizada         1
86             pluxee  Mail/formulario      Realizada         1
87            rituals  Mail/formulario      Realizada         1
88    samsung electro  Mail/formulario      Realizada         1
89  samsung telefonia  Mail/formulario      Realizada         1

[90 rows x 4 columns]


# ---------------------------------------------------------------------------------------------------------------------------

# **Extraer el canal de la columna SERVICIO para Evaluadas de eAlicia**

Ejecutar solo las dos primeras celdas, la última es para descargar el excel

In [129]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluaciones prueba Decathlon.xlsx'

# Cargar el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df.head()


,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. CALIDAD DE LA RESPUESTA.COND.LA RESPUESTA PROPORCIONADA DURANTE LA CONVERSACIÓN DE WHATSAPP ES FALSA. MUESTRA NULO CONOCIMIENTO (ENTENDIMIENTO) DE LA CONSULTA O PRODUCTO.,C13. CALIDAD DE LA RESPUESTA.COND.EL CHAT DE WHATSAPP NO HA SIDO ATENDIDO TRAS PASARSE EL PLAZO DE 5 MINUTOS Y NO RECIBIR RESPUESTA POR PARTE DE UN ASESOR.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA CON LA SOLUCIÓN A LA CONSULTA PLANTEADA POR EL CLIENTE.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA DIFERENTE A LO CONSULTA DESCRITA POR EL CLIENTE.,C15. GESTIÓN CORRECTA DE LA SOLICITUD,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR SÍ DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR NO DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR),C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.
0,101S_969-OP_11_291_20250226H_12_03,LLAMADA PRUEBA 1,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
1,101S_969-OP_11_291_20250226H_12_07,LLAMADA PRUEBA 2,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
2,101S_969-OP_11_291_20250226H_12_11,LLAMADA PRUEBA 3,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
3,101S_969-OP_11_291_20250226H_12_15,LLAMADA PRUEBA 4,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
4,101S_969-OP_11_291_20250226H_12_18,LLAMADA PRUEBA 5,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,


**De la columna servicio extraer SERVICIO**

In [130]:
# Lista de palabras clave
servicios_validos = ['Teléfono', 'Mail/Formulario', 'Web', 'Twitter', 'Facebook', 'Chat Web', 'Whatsapp', 'Chat']

# Mapeo de reemplazo para ajustar formato deseado
reemplazos = {
    'Teléfono': 'Telefono'
}

# Función para extraer la palabra clave válida
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return reemplazos.get(palabra, palabra)
    return None  # Retorna None si no encuentra ninguna palabra válida

# Aplicar la función a la columna "Servicio"
df['servicio_limpio'] = df['SERVICIO'].apply(limpiar_servicio)

# Mostrar el DataFrame con la nueva columna
print(df['servicio_limpio'])

0            Telefono
1            Telefono
2            Telefono
3            Telefono
4            Telefono
5            Telefono
6     Mail/Formulario
7     Mail/Formulario
8     Mail/Formulario
9     Mail/Formulario
10    Mail/Formulario
11    Mail/Formulario
12                Web
13                Web
14                Web
15                Web
16                Web
17                Web
18            Twitter
19            Twitter
20            Twitter
21           Facebook
22           Facebook
23           Facebook
24                Web
25                Web
26                Web
27           Whatsapp
28           Whatsapp
29           Whatsapp
30           Whatsapp
31           Whatsapp
32               Chat
Name: servicio_limpio, dtype: object


**Solo si se requiere descargar ejecutar la siguiente casilla**

In [131]:
# Guardar el DataFrame modificado como archivo Excel
#df.to_excel('Servicio-Limpio.xlsx', index=False)

# Descargar el archivo a tu computadora
#from google.colab import files
#files.download('Servicio-Limpio.xlsx')


# **Crear hoja REGISTRO para EVALUADAS**

Ejecutar solo la primera celda, la última es para descargar el excel

In [132]:
# Renombramos columnas para facilitar el trabajo
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Aplicamos el mapeo
df['tipo_contacto'] = df['servicio_limpio']

# Capitalizamos la primera letra de cada tipo_contacto
df['tipo_contacto'] = df['tipo_contacto'].str.capitalize()
df['tipo_contacto'] = df['tipo_contacto'].replace('Whatsapp', 'WhatsApp')

df['empresa'] = df['empresa'].str.lower().str.strip()

# Columna fija
df['tipo_de_acción'] = 'evaluada'

# Agrupamos por fecha, empresa y tipo_contacto (canal)
df = df.groupby(['empresa', 'tipo_contacto']).size().reset_index(name='cantidad')

# Añadimos columna de tipo_de_acción (ya está aplicada arriba, pero por si acaso)
df['tipo_de_acción'] = 'Evaluada'

# Reordenamos columnas
df = df[['empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]


# Mostramos resultado
df

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,decathlon,Chat,Evaluada,1
1,decathlon,Facebook,Evaluada,3
2,decathlon,Mail/formulario,Evaluada,6
3,decathlon,Telefono,Evaluada,6
4,decathlon,Twitter,Evaluada,3
5,decathlon,Web,Evaluada,9
6,decathlon,WhatsApp,Evaluada,5


Extraer hoja registro

In [133]:
# Guardar el DataFrame modificado como archivo Excel
#df.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
#from google.colab import files
#files.download('Registro.xlsx')

# -----------------------------------------------------------------------------------------------

# **Crear hoja Recuento por TIPO DE CONTACTO**

In [134]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Resumen Planificación.xlsx'

# Cargar el archivo Excel
df2 = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df2.head()

,Empresa,Categoría,Inicio,Fin,BBDD,Observaciones BBDD,Compra,Fecha Informe,Telefono,Mail/formulario,...,Twitter,Facebook,Chat,WhatsApp,Confirmación,Informe,Alta en eAlicia,Escenarios C3,Emisión (horas),Evaluación (horas)
0,SECTOR ALARMA,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,0,0,0,0,NaN,NaN,NaN,NaN,17.250000,25.875
1,ADT,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,...,0,0,0,0,No,NaN,NaN,NaN,5.716667,8.575
2,Securitas,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,...,0,0,0,0,NaN,NaN,NaN,NaN,5.716667,8.575
3,Prosegur,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,19,...,0,0,0,0,NaN,NaN,NaN,NaN,5.816667,8.725
4,Bruneau,Material de Oficina,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,0,0,12,0,NaN,NaN,NaN,NaN,18.650000,27.975


**Unir los dos excel, obteniendo evaluada, realizada y total**

In [135]:
# --- Paso 1: Unir evaluadas y realizadas ---
df_combined = pd.concat([df, df_final_realizada], ignore_index=True)

df_combined

,empresa,tipo_contacto,tipo_de_acción,cantidad
0,decathlon,Chat,Evaluada,1
1,decathlon,Facebook,Evaluada,3
2,decathlon,Mail/formulario,Evaluada,6
3,decathlon,Telefono,Evaluada,6
4,decathlon,Twitter,Evaluada,3
...,...,...,...,...
92,naturgy,Mail/formulario,Realizada,1
93,pluxee,Mail/formulario,Realizada,1
94,rituals,Mail/formulario,Realizada,1
95,samsung electro,Mail/formulario,Realizada,1


In [87]:
# --- Paso 2: Pivotear para tener columnas Evaluada y Realizada por canal ---
df_pivot = df_combined.pivot_table(index=['empresa', 'tipo_contacto'],
                                   columns='tipo_de_acción',
                                   values='cantidad',
                                   aggfunc='sum',
                                   fill_value=0).reset_index()

# Asegurar columnas consistentes
df_pivot = df_pivot.reindex(columns=['empresa', 'tipo_contacto', 'Evaluada', 'Realizada'], fill_value=0)

# --- Paso 3: Convertir df2 a formato largo ---
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']
df2_melt = df2.melt(id_vars=['Empresa', 'Inicio', 'Fin'],
                    value_vars=canales,
                    var_name='tipo_contacto',
                    value_name='Total')
df2_melt.rename(columns={'Empresa': 'empresa'}, inplace=True)
df2_melt['empresa'] = df2_melt['empresa'].str.lower().str.strip()

# --- Paso 4: Unir todo por empresa y canal ---
df_merged = pd.merge(df2_melt, df_pivot, on=['empresa', 'tipo_contacto'], how='left')
df_merged[['Evaluada', 'Realizada']] = df_merged[['Evaluada', 'Realizada']].fillna(0)

# --- Paso 5: Pivotear final: una columna por canal con sufijos ---
df_final = df_merged.pivot_table(index=['empresa', 'Inicio', 'Fin'],
                                 columns='tipo_contacto',
                                 values=['Evaluada', 'Realizada', 'Total'],
                                 aggfunc='sum',
                                 fill_value=0)

# Aplanar nombres de columnas
df_final.columns = [f"{canal}_{tipo}" for tipo, canal in df_final.columns]
df_final = df_final.reset_index()

# Estética: normalizar nombres
df_final['empresa'] = df_final['empresa'].str.lower().str.strip()

# --- Resultado final listo ---
df_final


,empresa,Inicio,Fin,Chat_Evaluada,Facebook_Evaluada,Mail/formulario_Evaluada,Telefono_Evaluada,Twitter_Evaluada,Web_Evaluada,WhatsApp_Evaluada,...,Twitter_Realizada,Web_Realizada,WhatsApp_Realizada,Chat_Total,Facebook_Total,Mail/formulario_Total,Telefono_Total,Twitter_Total,Web_Total,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12,6,55,135,6,6,0
1,adeslas,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,18,45,0,2,0
2,adt,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,18,45,0,2,0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,27,68,0,3,0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,55,135,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ups,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3,0,28,67,0,3,3
81,viatris,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,55,135,0,6,0
82,whirlpool,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,14,34,0,2,0
83,xiaomi,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,19,45,0,2,0


**Ordenar columnas para que esten juntos por canal**

In [136]:
# Paso final: ordenar columnas agrupadas por canal
orden_final = ['empresa', 'Inicio', 'Fin']
tipos = ['Realizada', 'Evaluada', 'Total']
for canal in canales:
    for tipo in tipos:
        col = f"{canal}_{tipo}"
        if col in df_final.columns:
            orden_final.append(col)

# Reordenar el DataFrame
df_final = df_final[orden_final]

df_final.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Facebook_Total,Twitter_Realizada,Twitter_Evaluada,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,4.0,0.0,135,1.0,0.0,55,0.0,...,6,0.0,0.0,6,0.0,0.0,12,0.0,0.0,0
1,adeslas,2025-05-05,2025-06-27,1.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


**--Generar una copia del df_final hasta aquí para luego trabajar el acumulativo--**

In [89]:
# Crear una copia profunda del DataFrame (independiente de la original) para trabajar con el acumulativo a posteriori
#df_copy = df_final.copy()

#print("Copia modificada")
#print(df_copy.head())


Copia modificada
       empresa     Inicio        Fin  Telefono_Realizada  Telefono_Evaluada  \
0         acer 2025-05-05 2025-06-27                 4.0                0.0   
1      adeslas 2025-05-05 2025-06-27                 1.0                0.0   
2          adt 2025-05-05 2025-06-27                 0.0                0.0   
3   aeda homes 2025-05-05 2025-06-27                 0.0                0.0   
4  aegon salud 2025-05-05 2025-06-27                 0.0                0.0   

   Telefono_Total  Mail/formulario_Realizada  Mail/formulario_Evaluada  \
0             135                        1.0                       0.0   
1              45                        0.0                       0.0   
2              45                        0.0                       0.0   
3              68                        0.0                       0.0   
4             135                        0.0                       0.0   

   Mail/formulario_Total  Web_Realizada  ...  Facebook_Total  \

**Obtener Total_general, total_evaluadas y total_realizadas**

In [137]:
# Identificamos columnas por tipo
cols_total = [col for col in df_final.columns if col.endswith('_Total')]
cols_realizada = [col for col in df_final.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_final.columns if col.endswith('_Evaluada')]

# Creamos las columnas de suma total
df_final['Total_general'] = df_final[cols_total].sum(axis=1)
df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)

df_final.head()


<ipython-input-137-d94e381a33ca>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_general'] = df_final[cols_total].sum(axis=1)
<ipython-input-137-d94e381a33ca>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)
<ipython-input-137-d94e381a33ca>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas
0,acer,2025-05-05,2025-06-27,4.0,0.0,135,1.0,0.0,55,0.0,...,6,0.0,0.0,12,0.0,0.0,0,220,5.0,0.0
1,adeslas,2025-05-05,2025-06-27,1.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,65,1.0,0.0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,65,0.0,0.0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,0.0,0.0,0,98,0.0,0.0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,0.0,0.0,0,196,0.0,0.0


**Obtener columnas por evaluar y por realizar según el total general**

In [138]:
df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']

df_final.head()

<ipython-input-138-257f9cce6d9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
<ipython-input-138-257f9cce6d9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_evaluar,Por_realizar
0,acer,2025-05-05,2025-06-27,4.0,0.0,135,1.0,0.0,55,0.0,...,0.0,12,0.0,0.0,0,220,5.0,0.0,220.0,215.0
1,adeslas,2025-05-05,2025-06-27,1.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,0.0,0.0,0,65,1.0,0.0,65.0,64.0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,0.0,0.0,0,65,0.0,0.0,65.0,65.0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0.0,0,0.0,0.0,0,98,0.0,0.0,98.0,98.0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,0.0,0.0,0,196,0.0,0.0,196.0,196.0


**Obtener los dias laborables restantes**

In [139]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_final['Fin'] = pd.to_datetime(df_final['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_final['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_final.head()


<ipython-input-139-3f456854cbaf>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Fin'] = pd.to_datetime(df_final['Fin'])
<ipython-input-139-3f456854cbaf>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_evaluar,Por_realizar,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,4.0,0.0,135,1.0,0.0,55,0.0,...,12,0.0,0.0,0,220,5.0,0.0,220.0,215.0,31
1,adeslas,2025-05-05,2025-06-27,1.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,65,1.0,0.0,65.0,64.0,31
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,65,0.0,0.0,65.0,65.0,31
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,98,0.0,0.0,98.0,98.0,31
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,196,0.0,0.0,196.0,196.0,31


**Calcular la cantidad de a realizar y evaluar según los días laborables restantes**

In [140]:
# Asegurarse de evitar división por cero o días negativos
df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
df_final['A_evaluar_por_dia'] = ((df_final['Total_general'] - df_final['Total_evaluadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)

df_final

<ipython-input-140-037b4bd358f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)
<ipython-input-140-037b4bd358f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_evaluar,Por_realizar,Dias_laborables_restantes,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,4.0,0.0,135,1.0,0.0,55,0.0,...,0.0,0,220,5.0,0.0,220.0,215.0,31,7,7
1,adeslas,2025-05-05,2025-06-27,1.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,65,1.0,0.0,65.0,64.0,31,2,2
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,65,0.0,0.0,65.0,65.0,31,2,2
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0.0,0,98,0.0,0.0,98.0,98.0,31,3,3
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,196,0.0,0.0,196.0,196.0,31,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ups,2025-05-05,2025-06-27,0.0,0.0,67,0.0,0.0,28,0.0,...,0.0,3,104,0.0,0.0,104.0,104.0,31,3,3
81,viatris,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,196,0.0,0.0,196.0,196.0,31,6,6
82,whirlpool,2025-05-05,2025-06-27,0.0,0.0,34,0.0,0.0,14,0.0,...,0.0,0,50,0.0,0.0,50.0,50.0,31,2,2
83,xiaomi,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,19,0.0,...,0.0,0,66,0.0,0.0,66.0,66.0,31,2,2


**Extraer el archivo final**

In [142]:
# Guardar el DataFrame modificado como archivo Excel
df_final.to_excel('Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ----------------------------------------------------------------------------------------------------

# Cargar un archivo del histórico distinto al último creado.

**Cuidado**, el bloque "Generar acumulativo..." debe ejecutarse manualmente punto a punto para que no tome el último  archivo creado del histórico sino este que estamos escogiendo manualmente.

In [141]:
import os
import glob
import pandas as pd

# Ruta base donde están los archivos históricos
ruta_base = "/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico"

# Buscar todos los archivos del historial
archivos = sorted(glob.glob(os.path.join(ruta_base, "Acumulativo_Recuento_historico_*.xlsx")))

# Mostrar lista con índice
print("📁 Archivos históricos disponibles:")
for i, archivo in enumerate(archivos):
    print(f"{i+1}. {os.path.basename(archivo)}")

# Elegir índice del archivo a cargar
indice = int(input("\n👉 Ingresá el número del archivo que querés cargar: ")) - 1

# Verificación
if 0 <= indice < len(archivos):
    archivo_seleccionado = archivos[indice]
    df_final_actualizado = pd.read_excel(archivo_seleccionado)
    print(f"\n✅ Archivo cargado: {archivo_seleccionado}")
else:
    print("❌ Índice fuera de rango.")


📁 Archivos históricos disponibles:
1. Acumulativo_Recuento_historico_2025-05-13_1.xlsx
2. Acumulativo_Recuento_historico_2025-05-13_2.xlsx
3. Acumulativo_Recuento_historico_2025-05-13_3.xlsx
4. Acumulativo_Recuento_historico_2025-05-13_4.xlsx
5. Acumulativo_Recuento_historico_2025-05-14_1.xlsx
6. Acumulativo_Recuento_historico_2025-05-14_2.xlsx


KeyboardInterrupt: Interrupted by user

# -----------------------------------------------------------------------------------------------------

# **Generar acumulativo para ver progreso del RECUENTO**

**Generar histórico acumaltivo**

**Solo ejecutar una vez!!**
Toma los valores de realizada y evaluada para ir actualizando los resultados.

In [95]:
# Limpiar espacios en nombres de columnas (importante antes del melt)
df2.columns = df2.columns.str.strip()

# También asegurarse que los valores en la columna 'Empresa' no tengan espacios
df2['Empresa'] = df2['Empresa'].str.strip()

# Verificamos que los nombres estén limpios y coincidan con 'canales'
canales = ['Telefono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']


In [96]:
# Filtrar columnas a actualizar
columnas_actualizar = [col for col in df_historico.columns if col.endswith('_Realizada') or col.endswith('_Evaluada')]

# Asegurarse de que los índices clave sean iguales antes de sumar
df_historico['empresa'] = df_historico['empresa'].str.strip().str.lower()
df_final['empresa'] = df_final['empresa'].str.strip().str.lower()

# Definir columnas clave
columnas_clave = ['empresa', 'Inicio', 'Fin']

# Hacemos merge para acumular
df_actualizado = pd.merge(
    df_historico,
    df_final[columnas_clave + columnas_actualizar],
    on=columnas_clave,
    how='outer',
    suffixes=('', '_nuevo')
)

# Sumar columnas *_Realizada y *_Evaluada
for col in columnas_actualizar:
    col_nuevo = f"{col}_nuevo"
    if col_nuevo in df_actualizado.columns:
        df_actualizado[col] = df_actualizado[col].fillna(0) + df_actualizado[col_nuevo].fillna(0)
        df_actualizado.drop(columns=col_nuevo, inplace=True)

# Seleccionar solo columnas *_realizada, *_evaluada, *_total y claves
columnas_filtradas = columnas_clave + [col for col in df_actualizado.columns if col.lower().endswith(('_realizada', '_evaluada', '_total'))]

df_actualizado = df_actualizado[columnas_filtradas]

df_actualizado

<ipython-input-96-754d7becf17a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['empresa'] = df_final['empresa'].str.strip().str.lower()


,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Facebook_Total,Twitter_Realizada,Twitter_Evaluada,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total
0,acer,2025-05-05,2025-06-27,24.0,0.0,135,6.0,0.0,55,0.0,...,6,0.0,0.0,6,0.0,0.0,12,0.0,0.0,0
1,adeslas,2025-05-05,2025-06-27,6.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ups,2025-05-05,2025-06-27,0.0,0.0,67,0.0,0.0,28,0.0,...,0,0.0,0.0,0,0.0,0.0,3,0.0,0.0,3
81,viatris,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
82,whirlpool,2025-05-05,2025-06-27,0.0,0.0,34,0.0,0.0,14,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
83,xiaomi,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,19,0.0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


**Obtener total general, evaluadas y realizadas**

In [97]:
# Identificamos columnas por tipo
cols_total = [col for col in df_actualizado.columns if col.endswith('_Total')]
cols_realizada = [col for col in df_actualizado.columns if col.endswith('_Realizada')]
cols_evaluada = [col for col in df_actualizado.columns if col.endswith('_Evaluada')]

# Creamos las columnas de suma total
df_actualizado['Total_general'] = df_actualizado[cols_total].sum(axis=1)
df_actualizado['Total_realizadas'] = df_actualizado[cols_realizada].sum(axis=1)
df_actualizado['Total_evaluadas'] = df_actualizado[cols_evaluada].sum(axis=1)

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Twitter_Total,Chat_Realizada,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas
0,acer,2025-05-05,2025-06-27,24.0,0.0,135,6.0,0.0,55,0.0,...,6,0.0,0.0,12,0.0,0.0,0,220,30.0,0.0
1,adeslas,2025-05-05,2025-06-27,6.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,65,6.0,0.0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,0.0,0.0,0,65,0.0,0.0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,0.0,0.0,0,98,0.0,0.0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,0.0,0.0,0,196,0.0,0.0


**Obtener columnas por evaluar y realizar**

In [98]:
df_actualizado['Por_realizar'] = df_actualizado['Total_general'] - df_actualizado['Total_realizadas']
df_actualizado['Por_evaluar'] = df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']

df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Evaluada,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar
0,acer,2025-05-05,2025-06-27,24.0,0.0,135,6.0,0.0,55,0.0,...,0.0,12,0.0,0.0,0,220,30.0,0.0,190.0,220.0
1,adeslas,2025-05-05,2025-06-27,6.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,0.0,0.0,0,65,6.0,0.0,59.0,65.0
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,0.0,0.0,0,65,0.0,0.0,65.0,65.0
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0.0,0,0.0,0.0,0,98,0.0,0.0,98.0,98.0
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,0.0,0.0,0,196,0.0,0.0,196.0,196.0


**Calcular días restantes**

In [99]:
import numpy as np
from datetime import datetime
from workalendar.europe.spain import Catalonia  # o Spain, SpainCatalonia, etc.

# Asegurar que la columna Fin está en formato datetime
df_actualizado['Fin'] = pd.to_datetime(df_actualizado['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_actualizado['Fin'].values.astype('datetime64[D]')

# Obtener feriados nacionales + Madrid del año actual y siguiente
cal = Catalonia()
año_actual = datetime.today().year
feriados = cal.holidays(año_actual) + cal.holidays(año_actual + 1)
feriados_np = np.array([np.datetime64(d[0], 'D') for d in feriados])

# Calcular días hábiles restantes excluyendo feriados
df_actualizado['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin, holidays=feriados_np)

# Visualizar
df_actualizado.head()

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,Chat_Total,WhatsApp_Realizada,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes
0,acer,2025-05-05,2025-06-27,24.0,0.0,135,6.0,0.0,55,0.0,...,12,0.0,0.0,0,220,30.0,0.0,190.0,220.0,31
1,adeslas,2025-05-05,2025-06-27,6.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,65,6.0,0.0,59.0,65.0,31
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0,0.0,0.0,0,65,0.0,0.0,65.0,65.0,31
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0,0.0,0.0,0,98,0.0,0.0,98.0,98.0,31
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0,0.0,0.0,0,196,0.0,0.0,196.0,196.0,31


**A realizar y a evaluar según los días restantes**

In [100]:
# Asegurarse de evitar división por cero o días negativos
df_actualizado['Dias_laborables_restantes'] = df_actualizado['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_actualizado['A_realizar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_realizadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)
df_actualizado['A_evaluar_por_dia'] = ((df_actualizado['Total_general'] - df_actualizado['Total_evaluadas']) / df_actualizado['Dias_laborables_restantes']).round(0).astype(int)

df_actualizado

,empresa,Inicio,Fin,Telefono_Realizada,Telefono_Evaluada,Telefono_Total,Mail/formulario_Realizada,Mail/formulario_Evaluada,Mail/formulario_Total,Web_Realizada,...,WhatsApp_Evaluada,WhatsApp_Total,Total_general,Total_realizadas,Total_evaluadas,Por_realizar,Por_evaluar,Dias_laborables_restantes,A_realizar_por_dia,A_evaluar_por_dia
0,acer,2025-05-05,2025-06-27,24.0,0.0,135,6.0,0.0,55,0.0,...,0.0,0,220,30.0,0.0,190.0,220.0,31,6,7
1,adeslas,2025-05-05,2025-06-27,6.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,65,6.0,0.0,59.0,65.0,31,2,2
2,adt,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,18,0.0,...,0.0,0,65,0.0,0.0,65.0,65.0,31,2,2
3,aeda homes,2025-05-05,2025-06-27,0.0,0.0,68,0.0,0.0,27,0.0,...,0.0,0,98,0.0,0.0,98.0,98.0,31,3,3
4,aegon salud,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,196,0.0,0.0,196.0,196.0,31,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ups,2025-05-05,2025-06-27,0.0,0.0,67,0.0,0.0,28,0.0,...,0.0,3,104,0.0,0.0,104.0,104.0,31,3,3
81,viatris,2025-05-05,2025-06-27,0.0,0.0,135,0.0,0.0,55,0.0,...,0.0,0,196,0.0,0.0,196.0,196.0,31,6,6
82,whirlpool,2025-05-05,2025-06-27,0.0,0.0,34,0.0,0.0,14,0.0,...,0.0,0,50,0.0,0.0,50.0,50.0,31,2,2
83,xiaomi,2025-05-05,2025-06-27,0.0,0.0,45,0.0,0.0,19,0.0,...,0.0,0,66,0.0,0.0,66.0,66.0,31,2,2


**Almacenar en Drive el histórico que alimentará al siguiente**

In [101]:
fecha = datetime.now().strftime("%Y-%m-%d")
contador = len(glob.glob(os.path.join(ruta_base, f"Acumulativo_Recuento_historico_{fecha}_*.xlsx")))
nombre_archivo = f"Acumulativo_Recuento_historico_{fecha}_{contador + 1}.xlsx"
ruta_guardado = os.path.join(ruta_base, nombre_archivo)

df_actualizado.to_excel(ruta_guardado, index=False)

print(f"✅ Histórico actualizado y guardado en: {ruta_guardado}")

✅ Histórico actualizado y guardado en: /content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Histórico/Acumulativo_Recuento_historico_2025-05-14_2.xlsx


**Descargar excel**

In [102]:
# Guardar el DataFrame modificado como archivo Excel
df_actualizado.to_excel('Acumulativo Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Acumulativo Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ------------------------------------------------------------------------------------------------------------------